<a href="https://colab.research.google.com/github/IrinaProkofieva/AutoML/blob/main/AutoML_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Установка зависимостей**

In [ ]:
!pip install -U lightautoml==0.3.8b1

In [ ]:
!pip install transformers[sentencepiece]
!pip install transformers[torch]

In [ ]:
!pip install optuna

In [ ]:
!pip install datasets

# **Импорты**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from lightautoml.automl.presets.text_presets import TabularNLPAutoML
from lightautoml.tasks import Task

from lightautoml.addons.interpretation import LimeTextExplainer, L2XTextExplainer
from lightautoml.report.report_deco import ReportDecoNLP

In [3]:
import shutil

import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.model_selection import train_test_split

import transformers
from transformers import pipeline
transformers.logging.set_verbosity(50)

import pickle
import torch
import string
import ast

In [4]:
from sklearn.base import BaseEstimator, ClassifierMixin
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import torch.nn.functional as F
import optuna
from datasets import load_metric, Dataset
from sklearn.ensemble import VotingClassifier

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **Достаем данные**

In [6]:
train_data = pd.read_csv('/content/drive/MyDrive/toxic_comments/jigsaw-toxic-comment-train.csv')
preprocessed_data = pd.read_csv('/content/drive/MyDrive/toxic_comments/preprocessed_train_data.csv')
val_data = pd.read_csv('/content/drive/MyDrive/toxic_comments/validation.csv')
test_data = pd.read_csv('/content/drive/MyDrive/toxic_comments/test.csv')
test_labels = pd.read_csv('/content/drive/MyDrive/toxic_comments/test_labels.csv')

In [7]:
submission = pd.read_csv('/content/drive/MyDrive/toxic_comments/sample_submission.csv')

Приведем данные к одному виду: добавим колонку с языком в тренировочный датасет, переименуем колонку с текстом в тестовой выборке.

In [8]:
train_data['lang'] = 'en'
test_data = test_data.rename(columns={"content": "comment_text"})

Добавим новые признаки для дальнейшей очистки данных

In [9]:
def enrich_data(data):
  data['words_count'] = data['comment_text'].apply(lambda x: len(x.split()))
  data['punctuation'] = data['comment_text'].apply(lambda x: len(list(filter(lambda c: c in string.punctuation, x))) / len(x))
  data['uppercase'] = data['comment_text'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x))

In [10]:
enrich_data(train_data)
enrich_data(preprocessed_data)

Очистим данные, удалим ненужные колонки

In [11]:
# Функция для очистки датасета
def clear_dataset(data):
  return data[(data['punctuation'] < 0.5) & (data['words_count'] < 250)][['comment_text', 'toxic']]

In [12]:
train_data = clear_dataset(train_data)
preprocessed_data = clear_dataset(preprocessed_data)

In [13]:
def save_submissions(preds, filename):
  submission['toxic'] = preds
  submission.to_csv('/content/drive/MyDrive/toxic_comments/'+filename, index = False)

# **LightAutoML подходы**

In [ ]:
roles = {
    'text': ['comment_text'],
    'target': 'toxic'
}

task = Task('binary')

### **Подход 1 - Linear + LGBM + CatBoost --- Random LSTM**

In [ ]:
automl_3 = TabularNLPAutoML(
    task = task,
    timeout=3600,
    cpu_limit=1,
    gpu_ids='0',
    reader_params={'n_jobs': 2, 'cv': 5, 'random_state': 42},
    general_params={'use_algos': [['linear_l2', 'lgb', 'lgb_tuned', 'cb', 'cb_tuned']]},
    tuning_params={'max_tuning_iter': 20, 'max_tuning_time': 30},
    text_params = {'lang': 'multi'},
    autonlp_params = {
        'model_name': 'random_lstm',
        'transformer_params': {
            'model_params': {
                'embed_size': 300, 'hidden_size': 256,
                'pooling': 'mean', 'num_layers': 1
                },
            'dataset_params': {'max_length': 128, 'embed_size': 300}
        }
    },
    memory_limit = 7
)

Оригинальный датасет (кросс-валидация)

In [ ]:
%%time
oof_pred_train_3 = automl_3.fit_predict(train_data.sample(frac=1), roles=roles, verbose = 10)
val_pred_train_3 = automl_3.predict(val_data)

[14:01:09] Stdout logging level is DEBUG.


INFO:lightautoml.automl.presets.base:Stdout logging level is DEBUG.


[14:01:10] Model language mode: multi


INFO3:lightautoml.automl.presets.text_presets:Model language mode: multi


[14:01:10] Task: binary



...

[14:25:20] Automl preset training completed in 1450.86 seconds



INFO:lightautoml.automl.presets.base:Automl preset training completed in 1450.86 seconds



[14:25:20] Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) 



INFO:lightautoml.automl.presets.base:Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) 



CPU times: user 21min 3s, sys: 3min 18s, total: 24min 22s
Wall time: 24min 12s


In [ ]:
not_nan = np.any(~np.isnan(oof_pred_train_3.data), axis=1)

print('Check scores:')
print('Train scores: {}'.format(roc_auc_score(train_data[roles['target']].values[not_nan], oof_pred_train_3.data[not_nan][:, 0])))
print('Validation scores: {}'.format(roc_auc_score(val_data[roles['target']].values, val_pred_train_3.data[:, 0])))

Check scores:
Train scores: 0.49961716872267836
Validation scores: 0.5944670413469275


In [ ]:
test_pred_train_3 = automl_3.predict(test_data)
print('Test scores: {}'.format(roc_auc_score(test_labels[roles['target']].values, test_pred_train_3.data[:, 0])))

Test scores: 0.5885277987183342


In [ ]:
save_submissions(test_pred_train_3, 'automl_based_train.csv')

Оригинальный датасет (явно указываем валидационную выборку)

In [ ]:
%%time
oof_pred_train_3_1 = automl_3.fit_predict(train_data.sample(frac=1), roles=roles, verbose = 10, valid_data=val_data)
val_pred_train_3_1 = automl_3.predict(val_data)

[14:26:15] Stdout logging level is DEBUG.


INFO:lightautoml.automl.presets.base:Stdout logging level is DEBUG.


[14:26:15] Model language mode: multi


INFO3:lightautoml.automl.presets.text_presets:Model language mode: multi


[14:26:15] Task: binary



...

[14:36:27] Model description:
Final prediction for new objects (level 0) = 
	 0.65032 * (1 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.21308 * (1 averaged models Lvl_0_Pipe_1_Mod_0_LightGBM) +
	 0.13660 * (1 averaged models Lvl_0_Pipe_1_Mod_3_CatBoost) 



INFO:lightautoml.automl.presets.base:Model description:
Final prediction for new objects (level 0) = 
	 0.65032 * (1 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.21308 * (1 averaged models Lvl_0_Pipe_1_Mod_0_LightGBM) +
	 0.13660 * (1 averaged models Lvl_0_Pipe_1_Mod_3_CatBoost) 

100%|██████████| 8/8 [00:04<00:00,  1.78it/s]


[14:36:33] Feature concated__comment_text transformed


INFO3:lightautoml.transformers.text:Feature concated__comment_text transformed


CPU times: user 9min 36s, sys: 18.5 s, total: 9min 54s
Wall time: 10min 18s


In [ ]:
print('Check scores:')
print('Validation scores: {}'.format(roc_auc_score(val_data[roles['target']].values, val_pred_train_3_1.data[:, 0])))

Check scores:
Validation scores: 0.6215929915576852


In [ ]:
test_pred_train_3_1 = automl_3.predict(test_data)
print('Test scores: {}'.format(roc_auc_score(test_labels[roles['target']].values, test_pred_train_3_1.data[:, 0])))

100%|██████████| 63/63 [00:41<00:00,  1.52it/s]


[14:42:13] Feature concated__comment_text transformed


INFO3:lightautoml.transformers.text:Feature concated__comment_text transformed


Test scores: 0.614309527093237


In [ ]:
save_submissions(test_pred_train_3_1, 'automl_based_train_1.csv')

Предобработанные данные

In [ ]:
%%time
oof_pred_preprocessed_3 = automl_3.fit_predict(preprocessed_data.sample(frac=1), roles=roles, verbose = 10)
val_pred_preprocessed_3 = automl_3.predict(val_data)

[14:44:52] Stdout logging level is DEBUG.


INFO:lightautoml.automl.presets.base:Stdout logging level is DEBUG.


[14:44:52] Model language mode: multi


INFO3:lightautoml.automl.presets.text_presets:Model language mode: multi


[14:44:52] Task: binary



...

[14:49:20] Model description:
Final prediction for new objects (level 0) = 
	 0.21345 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.40460 * (5 averaged models Lvl_0_Pipe_1_Mod_0_LightGBM) +
	 0.27817 * (5 averaged models Lvl_0_Pipe_1_Mod_2_CatBoost) +
	 0.10378 * (5 averaged models Lvl_0_Pipe_1_Mod_3_CatBoost) 



INFO:lightautoml.automl.presets.base:Model description:
Final prediction for new objects (level 0) = 
	 0.21345 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.40460 * (5 averaged models Lvl_0_Pipe_1_Mod_0_LightGBM) +
	 0.27817 * (5 averaged models Lvl_0_Pipe_1_Mod_2_CatBoost) +
	 0.10378 * (5 averaged models Lvl_0_Pipe_1_Mod_3_CatBoost) 

100%|██████████| 8/8 [00:07<00:00,  1.06it/s]


[14:49:29] Feature concated__comment_text transformed


INFO3:lightautoml.transformers.text:Feature concated__comment_text transformed


CPU times: user 4min 37s, sys: 44.6 s, total: 5min 21s
Wall time: 4min 43s


In [ ]:
not_nan = np.any(~np.isnan(oof_pred_preprocessed_3.data), axis=1)

print('Check scores:')
print('Train scores: {}'.format(roc_auc_score(preprocessed_data[roles['target']].values, oof_pred_preprocessed_3.data[:, 0])))
print('Validation scores: {}'.format(roc_auc_score(val_data[roles['target']].values, val_pred_preprocessed_3.data[:, 0])))

Check scores:
Train scores: 0.4724856324754072
Validation scores: 0.5782191279076749


**Итого:**

- Оригинальный датасет, кросс-валидация
  - *OOF scores*: 0.49961716872267836
  - *Validation scores*: 0.5944670413469275
  - *Test scores*: 0.5885277987183342

- Оригинальный датасет, явно указана валидационная выборка
  - *Validation scores*: **0.6215929915576852**
  - *Test scores*: **0.614309527093237**

- Предобработанные данные
  - *Train scores*: 0.4724856324754072
  - *Validation scores*: 0.5782191279076749

### **Подход 2 - BERT**

In [ ]:
automl = TabularNLPAutoML(
    task=task,
    timeout=3600,
    gpu_ids='0',
    reader_params={'cv': 5, 'random_state': 42},
    general_params={
        'nested_cv': False,
        'use_algos': [['nn']]
    },
    autonlp_params={
        'sent_scaler': 'l2'
    },
    text_params={
        'lang': 'multi',
        'bert_model': 'bert-base-multilingual-cased'
    },
    nn_params={
        'opt_params': {'lr': 1e-3},
        'max_length': 128,
        'bs': 32,
        'n_epochs': 7
    },
    memory_limit = 7
)

Обучающая выборка

In [ ]:
%%time
oof_pred_train_1 = automl.fit_predict(train_data.sample(frac=1), roles=roles, verbose = 10, valid_data=val_data)
val_pred_train_1 = automl.predict(val_data)

[18:51:54] Stdout logging level is DEBUG.


INFO:lightautoml.automl.presets.base:Stdout logging level is DEBUG.


[18:51:54] Model language mode: multi


INFO3:lightautoml.automl.presets.text_presets:Model language mode: multi


[18:51:54] Task: binary



...

[22:29:34] Automl preset training completed in 13059.14 seconds



INFO:lightautoml.automl.presets.base:Automl preset training completed in 13059.14 seconds



[22:29:34] Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (1 averaged models Lvl_0_Pipe_0_Mod_0_TorchNN__linear_layer_0) 



INFO:lightautoml.automl.presets.base:Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (1 averaged models Lvl_0_Pipe_0_Mod_0_TorchNN__linear_layer_0) 



CPU times: user 3h 28min 30s, sys: 41 s, total: 3h 29min 11s
Wall time: 3h 38min 33s


In [ ]:
print('Validation scores: {}'.format(roc_auc_score(val_data[roles['target']].values, val_pred_train_1.data[:, 0])))

Validation scores: 0.5010904156308919


In [ ]:
test_pred_train_1 = automl.predict(test_data)
print('Test scores: {}'.format(roc_auc_score(test_labels[roles['target']].values, test_pred_train_1.data[:, 0])))

Test scores: 0.4967793632103666


In [ ]:
save_submissions(test_pred_train_1, 'automl_bert_initial_b.csv')

Предобработанные данные

**Итого:**

Оригинальный датасет:
 - Validation scores: 0.5010904156308919
 - Test scores: 0.4967793632103666

### **Подход 3 - Linear + Bert --- BERT Pooling**

In [ ]:
automl_1 = TabularNLPAutoML(
    task=task,
    timeout=3600,
    memory_limit = 7,
    general_params={'nested_cv': False, 'use_algos': [['linear_l2', 'nn']]},
    text_params={'lang': 'multi', 'bert_model': 'bert-base-multilingual-cased'},
    reader_params={'cv': 5},
    linear_pipeline_params={'text_features': 'embed'},
    autonlp_params={
        'model_name': 'pooled_bert',
        'transformer_params': {
            'model_params': {'pooling': 'mean'},
            'dataset_params': {'max_length': 128},
            'loader_params': {
                'batch_size': 32,
                'num_workers': 2
             }
        }
    },
    nn_params={
        'opt_params': {'lr': 3e-5},
        'lang': 'multi',
        'path_to_save': '/content/drive/MyDrive/toxic_comments/models',
        'bert_name': 'bert-base-multilingual-cased',
        'pooling': 'mean',
        'max_length': 128,
        'bs': 32,
        'n_epochs': 10
    },
)

Оригинальный датасет

In [ ]:
%%time
val_pred_train_2 = automl_1.fit_predict(train_data.sample(frac=1), roles=roles, verbose = 10, valid_data=val_data)

[14:54:59] Stdout logging level is DEBUG.


INFO:lightautoml.automl.presets.base:Stdout logging level is DEBUG.


[14:54:59] Model language mode: multi


INFO3:lightautoml.automl.presets.text_presets:Model language mode: multi


[14:54:59] Task: binary



...

[17:56:18] Automl preset training completed in 10879.25 seconds



INFO:lightautoml.automl.presets.base:Automl preset training completed in 10879.25 seconds



[17:56:18] Model description:
Final prediction for new objects (level 0) = 
	 0.32581 * (1 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.67419 * (1 averaged models Lvl_0_Pipe_1_Mod_0_TorchNN__linear_layer_0) 



INFO:lightautoml.automl.presets.base:Model description:
Final prediction for new objects (level 0) = 
	 0.32581 * (1 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.67419 * (1 averaged models Lvl_0_Pipe_1_Mod_0_TorchNN__linear_layer_0) 



CPU times: user 2h 41min 36s, sys: 52.7 s, total: 2h 42min 28s
Wall time: 3h 1min 19s


In [ ]:
print('Check scores. Initial')
print('Validation scores: {}'.format(roc_auc_score(val_data[roles['target']].values, val_pred_train_2.data[:, 0])))

Check scores. Initial
Validation scores: 0.8399286666426486


In [ ]:
test_pred_train_2 = automl_1.predict(test_data)
print('Test scores: {}'.format(roc_auc_score(test_labels[roles['target']].values, test_pred_train_2.data[:, 0])))

100%|██████████| 1995/1995 [07:37<00:00,  4.36it/s]


[18:03:59] Feature concated__comment_text transformed


INFO3:lightautoml.transformers.text:Feature concated__comment_text transformed


Test scores: 0.8307054600643403


In [ ]:
save_submissions(test_pred_train_2, 'automl_bert_lin_train.csv')

**Итого**

Оригинальный датасет, явное указание валидационной выборки
  - *Validation scores*: **0.8399286666426486**
  - *Test scores*: **0.8307054600643403**

### **Выводы**

- Лучший скор на тестовой выборке: **0.83**.
- Подход с комбинацией Linear и BERT дал скор значительно лучший, чемдва остальных подхода.
- Обучение на небольшом датасете, который более похож на валидационную и тестовую выборки, чем оригинальный большой датасет, не дает преимуществ.

# **Не LightAutoML подходы**

### Ансамблирование трансформеров

С использованием подбора гиперпараметров через optuna

In [15]:
metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

class HuggingFaceClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, model_name='bert-base-uncased', num_labels=2):
        self.model_name = model_name
        self.num_labels = num_labels
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def objective(self, trial, model, tokenizer, train_dataset, val_dataset):
        # Предложение гиперпараметров
        learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-4)

        # Определение аргументов тренировки
        training_args = TrainingArguments(
            output_dir='/content/drive/MyDrive/toxic_comments/opt',
            learning_rate=learning_rate,
            num_train_epochs=3,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=32,
            evaluation_strategy="epoch",
        )

        # Инициализация Trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=compute_metrics
        )

        # Обучение модели
        trainer.train()

        # Вычисление и возврат метрики для оптимизации
        eval_result = trainer.evaluate()
        metric_to_optimize = eval_result["eval_accuracy"]

        return metric_to_optimize

    def fit(self, X, y):
        train_data_new = X[X['is_val'] == False]
        val_data_new = X[X['is_val']]

        train_data_new.loc[:, ('label')] = y[:len(train_data_new)]
        train_inputs = self.tokenizer(train_data_new['comment_text'].tolist(), padding=True, truncation=True, return_tensors="pt")
        train_data_new['input_ids'] = pd.Series(train_inputs['input_ids'].tolist())
        train_data_new['attention_mask'] = pd.Series(train_inputs['attention_mask'].tolist())
        tds = Dataset.from_pandas(train_data_new)

        val_data_new.loc[:, ('label')] = y[len(train_data_new):]
        val_inputs = self.tokenizer(val_data_new['comment_text'].tolist(), padding=True, truncation=True, return_tensors="pt")
        val_data_new['input_ids'] = pd.Series(val_inputs['input_ids'].tolist())
        val_data_new['attention_mask'] = pd.Series(val_inputs['attention_mask'].tolist())
        vds = Dataset.from_pandas(train_data_new)

        model = AutoModelForSequenceClassification.from_pretrained(self.model_name, num_labels=self.num_labels)

        # Создание study object и запуск оптимизации
        study = optuna.create_study(direction='maximize')
        study.optimize(lambda trial: self.objective(trial, model, self.tokenizer, Dataset.from_pandas(train_data_new.sample(frac=1)), vds), n_trials=2)

        # Получение лучших гиперпараметров
        best_params = study.best_params
        print(f"Best parameters for {self.model_name}: {best_params}")

        print(f"Training {self.model_name}...")

        training_args = TrainingArguments(
            output_dir='/content/drive/MyDrive/toxic_comments/results',
            learning_rate=best_params['learning_rate'],
            num_train_epochs=3,
            per_device_train_batch_size=8,
            warmup_steps=500,
            weight_decay=0.01,
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tds,
            eval_dataset=vds,
        )

        trainer.train()

        self.model = model

        return self

    def predict(self, data):
        inputs = self.tokenizer(data['comment_text'].tolist(), padding=True, truncation=True, return_tensors="pt")

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = self.model.to(device)
        inputs = inputs.to(device)

        with torch.no_grad():
            outputs = model(**inputs)

        predictions = torch.argmax(outputs.logits, dim=-1).cpu().numpy()

        return predictions

    def predict_proba(self, data):
        inputs = self.tokenizer(data['comment_text'].tolist(), padding=True, truncation=True, return_tensors="pt")

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = self.model.to(device)
        inputs = inputs.to(device)

        with torch.no_grad():
            outputs = model(**inputs)

        probabilities = F.softmax(outputs.logits, dim=-1)

        probabilities = probabilities.cpu().numpy()

        return probabilities


<ipython-input-15-ea5a79d15d00>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [15]:
pd.options.mode.chained_assignment = None

classifiers = [
    ('model1', HuggingFaceClassifier(model_name='distilbert-base-multilingual-cased')),
    ('model2', HuggingFaceClassifier(model_name='roberta-base'))
]

ensemble = VotingClassifier(estimators=classifiers, voting='soft')

In [16]:
# train_data['is_val'] = False
preprocessed_data['is_val'] = False
val_data['is_val'] = True
new_data = pd.concat([preprocessed_data, val_data], ignore_index=True)

ensemble.fit(new_data[['comment_text', 'is_val']], new_data['toxic'])

INFO:optuna.storages._in_memory:A new study created in memory with name: no-name-b0f825ac-240f-487b-a7a2-e9b5877a18e6
<ipython-input-14-ea5a79d15d00>:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-4)


{'eval_loss': 0.1300072968006134, 'eval_accuracy': 0.9560260586319218, 'eval_runtime': 28.876, 'eval_samples_per_second': 63.79, 'eval_steps_per_second': 2.009, 'epoch': 1.0}
{'eval_loss': 0.027239393442869186, 'eval_accuracy': 0.992942453854506, 'eval_runtime': 29.8931, 'eval_samples_per_second': 61.62, 'eval_steps_per_second': 1.94, 'epoch': 2.0}
{'loss': 0.2082, 'learning_rate': 9.155073731486218e-06, 'epoch': 2.16}
{'eval_loss': 0.002906771143898368, 'eval_accuracy': 0.998914223669924, 'eval_runtime': 29.3794, 'eval_samples_per_second': 62.697, 'eval_steps_per_second': 1.974, 'epoch': 3.0}
{'train_runtime': 375.6408, 'train_samples_per_second': 14.711, 'train_steps_per_second': 1.845, 'train_loss': 0.1558948474873024, 'epoch': 3.0}


INFO:optuna.study.study:Trial 0 finished with value: 0.998914223669924 and parameters: {'learning_rate': 3.2872881326010096e-05}. Best is trial 0 with value: 0.998914223669924.


{'eval_loss': 0.002906771143898368, 'eval_accuracy': 0.998914223669924, 'eval_runtime': 29.6334, 'eval_samples_per_second': 62.16, 'eval_steps_per_second': 1.957, 'epoch': 3.0}


<ipython-input-14-ea5a79d15d00>:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-4)


...

<ipython-input-14-ea5a79d15d00>:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-4)


{'eval_loss': 0.33383628726005554, 'eval_accuracy': 0.9277958740499457, 'eval_runtime': 54.1059, 'eval_samples_per_second': 34.044, 'eval_steps_per_second': 1.072, 'epoch': 1.0}
{'eval_loss': 0.10333548486232758, 'eval_accuracy': 0.9641693811074918, 'eval_runtime': 54.0499, 'eval_samples_per_second': 34.08, 'eval_steps_per_second': 1.073, 'epoch': 2.0}
{'loss': 0.2657, 'learning_rate': 1.3396821463777511e-05, 'epoch': 2.16}
{'eval_loss': 0.0512084923684597, 'eval_accuracy': 0.9858849077090119, 'eval_runtime': 54.097, 'eval_samples_per_second': 34.05, 'eval_steps_per_second': 1.072, 'epoch': 3.0}
{'train_runtime': 672.313, 'train_samples_per_second': 8.219, 'train_steps_per_second': 1.031, 'train_loss': 0.22649654371913894, 'epoch': 3.0}


INFO:optuna.study.study:Trial 1 finished with value: 0.9858849077090119 and parameters: {'learning_rate': 4.8103612820714066e-05}. Best is trial 1 with value: 0.9858849077090119.


{'eval_loss': 0.0512084923684597, 'eval_accuracy': 0.9858849077090119, 'eval_runtime': 54.4342, 'eval_samples_per_second': 33.839, 'eval_steps_per_second': 1.066, 'epoch': 3.0}
Best parameters for roberta-base: {'learning_rate': 4.8103612820714066e-05}
Training roberta-base...
{'loss': 0.1625, 'learning_rate': 4.8103612820714066e-05, 'epoch': 2.16}
{'train_runtime': 504.8326, 'train_samples_per_second': 10.946, 'train_steps_per_second': 1.373, 'train_loss': 0.16237492031521267, 'epoch': 3.0}


VotingClassifier(estimators=[('model1',
                              HuggingFaceClassifier(model_name='distilbert-base-multilingual-cased')),
                             ('model2',
                              HuggingFaceClassifier(model_name='roberta-base'))],
                 voting='soft')

In [18]:
pickle.dump(ensemble, open('/content/drive/MyDrive/toxic_comments/ensemble_model.pkl', 'wb'))

In [16]:
ensemble = pickle.load(open('/content/drive/MyDrive/toxic_comments/ensemble_model.pkl', 'rb'))

In [18]:
predictions = ensemble.predict_proba(val_data)

In [22]:
print('Check scores')
print('Validation scores: {}'.format(roc_auc_score(val_data['toxic'].values, predictions[:, 0])))

Check scores
Validation scores: 0.9087164536742443


In [21]:
test_preds = ensemble.predict_proba(test_data)
print('Test scores: {}'.format(roc_auc_score(test_labels['toxic'].values, test_preds[:, 0])))

Test scores: 0.8843136524336524


**Итого:**
- *Validation scores*: 0.9087164536742443
- *Test scores*: 0.8843136524336524

### **Выводы**
Получилось обогнать LightAutoML за счет ансамблирования двух трансформеров, но такой подход оказался гораздо более требовательным к ресурсам, из-за чего пришлось обучать на предобработанном датасете меньшего размера и с небольшим числом подбираемых с помощью optuna гиперпараметров. LightAutoML действительно хорош для быстрого и относительно простого достижения хороших результатов!

Лучший скор на тестовой выборке **0.88**.